<a href="https://colab.research.google.com/github/GeomaticsCaminosUPM/footprint_attributes/blob/main/examples/example_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Footprint Attributes package example

Install libraries

In [1]:
!pip install geopandas
!pip install git+https://github.com/GeomaticsCaminosUPM/footprint_attributes.git
!pip install folium matplotlib mapclassify

  Cloning https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to /tmp/pip-req-build-ij10keoe
  Running command git clone --filter=blob:none --quiet https://github.com/GeomaticsCaminosUPM/footprint_attributes.git /tmp/pip-req-build-ij10keoe
  Resolved https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to commit b0529d807b30318fc8c6f5499c9c77d76b4e5a51
  Preparing metadata (setup.py) ... done
  Created wheel for footprint_attributes: filename=footprint_attributes-0.3.0-py3-none-any.whl size=16678 sha256=87c24cfa04a9cf020028ff020b3c2e08312a6c8a478bc71b7a89487eb89c14f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-bchh5_t7/wheels/7f/bd/c1/7a442bce743433d970ec52e18518fa8d82cb93ef44a7babcbf
Successfully built footprint_attributes
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.5 MB/s eta 0:00:00


In [2]:
import footprint_attributes
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/footprint_attributes

In [3]:
help(footprint_attributes.get_forces_gdf)

Help on function get_forces_gdf:

get_forces_gdf(geoms: geopandas.geodataframe.GeoDataFrame, buffer: float = 0, height_column: str = None, min_radius: float = 0) -> geopandas.geodataframe.GeoDataFrame
    Calculates force-based metrics for building footprints based on their geometry and proximity.
    
    Parameters:
        geoms (gpd.GeoDataFrame): A GeoDataFrame containing building footprints as polygon geometries.
        buffer (float): Buffer distance in meters to determine if two buildings are considered touching.
        height_column (str, optional): Column name in `geoms` specifying the building height in meters.
                                       If `None`, all buildings are assumed to have a uniform height of 1.
        min_radius (float, optional): Minimum distance multiplier used to exclude forces that would otherwise 
                                        increase momentum. Forces with a distance below a threshold 
                                        (`min_rad

## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [3]:
!wget https://github.com/GeomaticsCaminosUPM/footprint_attributes/raw/refs/heads/main/examples/footprints.gpkg

--2025-02-08 22:06:15--  https://github.com/GeomaticsCaminosUPM/footprint_attributes/raw/refs/heads/main/examples/footprints.gpkg
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GeomaticsCaminosUPM/footprint_attributes/refs/heads/main/examples/footprints.gpkg [following]
--2025-02-08 22:06:15--  https://raw.githubusercontent.com/GeomaticsCaminosUPM/footprint_attributes/refs/heads/main/examples/footprints.gpkg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 372736 (364K) [application/octet-stream]
Saving to: ‘footprints.gpkg’

footprints.gpkg     100%[===================>] 364.00K  --.-KB/s    in

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [4]:
footprints_gdf = gpd.read_file('/content/footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [5]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

There are multiplart geometries.


Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [6]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,gid,geometry
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


## 2. Relative position


Determines if the building touches other structures (relative position in the city block). This is done by calculating "forces" that neighboring structures exert on the building. The force is proportional to the contact area (length of touching footprints multiplied by building height) in the normal direction of the touching plane.


Lets calculate the forces with `get_forces_gdf()`

In [8]:
forces = footprint_attributes.get_forces_gdf(footprints_gdf,height_column=None,buffer=0.2,min_radius=0.5) #20 cm of buffer
forces

,height,force,confinement_ratio,angular_acc,angle,geometry
0,1.0,0.697807,2.119091,0.320945,2.465851e+00,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1.0,0.641808,0.766662,1.613179,1.159873e+00,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,1.0,0.341574,7.681600,0.001463,2.060583e-01,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,1.0,0.020213,179.790671,0.067755,2.266605e+00,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,1.0,0.287717,0.000000,0.220567,0.000000e+00,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...,...,...,...
939,1.0,1.113140,0.000000,0.034521,0.000000e+00,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,1.0,0.791168,0.000000,0.220128,4.214685e-08,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,1.0,0.977905,1.111545,1.164580,1.587350e+00,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,1.0,0.769893,1.679563,0.817435,1.745959e+00,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


Now determine the relative position using the forces calculated before with `relative_position()`

In [9]:
footprints_gdf['relative_position'] = footprint_attributes.relative_position(forces,min_angular_acc=2.133,min_confinement=1,min_angle=0.78,min_force=0.1666)
footprints_gdf

,index,gid,geometry,relative_position
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1...",confined
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1...",corner
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1...",confined
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1...",confined
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1...",lateral
...,...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1...",lateral
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1...",lateral
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1...",confined
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1...",confined


We create a new column `"relative_position"` that classifies buildings into the following categories:
  1. **"torque"**: Buildings of class **confined** or **corner** with an angular acceleration exceeding the minimum.
  2. **"confined"**: Structures touching on both the left and right lateral sides.
  3. **"corner"**: Structures touching at a corner (determined by force and angle thresholds).
  4. **"lateral"**: Structures touching on either the left or right side.
  5. **"isolated"**: No touching structures.

## 3. Irregularity


Irregularity meassured following international standards

In [7]:
result = footprint_attributes.get_eurocode_8_irregularity(footprints_gdf)
result

,excentricity_ratio,radius_ratio,slenderness,compactness,angle_excentricity,angle_slenderness
0,0.245076,0.349005,1.232601,1.000000,7.401332,75.086654
1,0.008968,0.163555,3.192047,0.999683,73.967503,86.104788
2,0.044199,0.279863,2.961852,0.999880,44.943716,65.468554
3,0.016334,0.218771,3.338191,0.997752,-41.933803,4.542072
4,0.259211,0.221460,2.101109,0.859100,7.573183,10.539551
...,...,...,...,...,...,...
939,0.635795,0.209521,1.100164,0.909876,-52.563560,6.732787
940,0.011709,0.282438,1.158917,1.000000,-36.565814,12.724535
941,0.185631,0.340129,1.739688,0.875405,44.692859,76.976761
942,0.158282,0.439820,1.350890,0.893873,76.603246,62.699325


In [8]:
footprints_gdf['excentricity_ratio_EC8'] = result['excentricity_ratio']
footprints_gdf['radius_ratio_EC8'] = result['radius_ratio']
footprints_gdf['slenderness_EC8'] = result['slenderness']
footprints_gdf['compactness_EC8'] = result['compactness']

In [9]:
result = footprint_attributes.get_costa_rica_irregularity(footprints_gdf)
result

,excentricity_ratio,angle
0,0.032608,8.479209
1,0.001232,86.251629
2,0.009637,56.560882
3,0.002224,-34.108901
4,0.040643,8.834586
...,...,...
939,0.054849,-52.578779
940,0.001424,-36.054181
941,0.032035,48.672230
942,0.030572,77.228576


In [10]:
footprints_gdf['excentricity_ratio_CR'] = result['excentricity_ratio']

In [11]:
result = footprint_attributes.get_mexico_NTC_irregularity(footprints_gdf)
result

,setback_ratio,hole_ratio
0,0.000000,0.0
1,0.019953,0.0
2,0.020580,0.0
3,0.066400,0.0
4,0.427917,0.0
...,...,...
939,0.245702,0.0
940,0.000000,0.0
941,0.298848,0.0
942,0.364713,0.0


In [12]:
footprints_gdf['setback_ratio_NTC'] = result['setback_ratio']
footprints_gdf['hole_ratio_NTC'] = result['hole_ratio']

In [13]:
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] <= 0.3].explore(color='green')
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] > 0.3].explore(m=m,color='red')
m

## 3. Irregularity



- **Convex hull **momentum** Irregularity Index**: Our own index to quantify the irregularity of building footprints.
  - Formula: $\frac{l \cdot d}{L}$, where:
    - $l$: Length of the shapes outside the convex hull.
    - $d$: Distance of the center of gravity of the shapes outside the hull to the hull.
    - $L$: Total length of the convex hull.

In [10]:
footprints_gdf['convex_hull_momentum'] = footprint_attributes.convex_hull_momentum(footprints_gdf)
footprints_gdf

,index,gid,geometry,relative_position,convex_hull_momentum
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1...",confined,0.000000
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1...",corner,0.000148
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1...",confined,0.000000
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1...",confined,0.004862
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1...",lateral,0.463150
...,...,...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1...",lateral,0.721346
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1...",lateral,0.000000
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1...",confined,0.338223
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1...",confined,0.296966


- **Polsby-Popper Index**: A measure of shape compactness (similarity to a circle).
   - Formula: $\text{Polsby-Popper Index} = \frac{4 \pi A}{P^2}$
    where:
      - $A$: Area of the polygon.
      - $P$: Perimeter of the polygon.


In [ ]:
footprints_gdf['polsby_popper'] = footprint_attributes.polsby_popper(footprints_gdf,fill_holes=False)
footprints_gdf

,index,gid,geometry,relative_position,shape_irregularity,polsby_popper
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1...",confined,0.000000,0.731584
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1...",corner,0.000148,0.570544
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1...",confined,0.000000,0.615186
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1...",confined,0.004862,0.556486
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1...",lateral,0.463150,0.554642
...,...,...,...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1...",lateral,0.721346,0.562271
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1...",lateral,0.000000,0.780750
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1...",confined,0.338223,0.563278
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1...",confined,0.296966,0.642602


- **Inertia Irregularity**: Under development

## 4. Example code to plot results


Plot a map (needs pip install folium matplotlib mapclassify)

In [ ]:
footprints_gdf.explore(
    column='inertia_slenderness',       # Column to base the color on
    cmap='YlOrRd',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    #vmin = 1.8,
    #vmax = 2.5               # Set max and min values if needed
)

In [ ]:
m=footprints_gdf.geometry.explore(color='gray')
m=footprints_gdf.geometry.centroid.explore(m=m)
m=footprints_gdf.geometry.boundary.centroid.explore(m=m, color='red')
m

## 5. Save results

In [ ]:
footprints_gdf.to_file('path/to/save/file.gpkg')